In [ ]:
using ITensors
using ITensorMPS
using LinearAlgebra
using PolyChaos
using QuadGK
using Plots

In [ ]:
function spectral_func(input::AbstractString, D::Float64, g::Float64)
    inband = x -> (-D <= x <= D)
        
    if input == "flat"
        J = x -> inband(x) ? 1/(2D) : 0.0
    elseif input == "elliptical"
        J = x -> inband(x) ? sqrt(1 - (x/D)^2) : 0.0
    elseif input == "ohmic"
        J = x -> inband(x) ? abs(x) : 0.0
    elseif input == "lorentzian"
        J = x -> 1/(1 + (x/D)^2)
    else
        error("spectral function type not recognized")
    end
    # normalization
    norm = quadgk(J, -D, D)[1]           
    Jnorm = x -> g*D/pi * J(x) / norm
    return Jnorm
end

function thermofield_transform(J, beta::Float64, mu::Float64) #spectral function, inverse temp, chemical potential
    """thermofield purification using fermi function ancilla"""
    fermi(k) = 1/(1 + exp(beta*k - beta*mu))
    J1 = w -> J(w) * fermi(w) #filled mode spectral density
    J2 = w -> J(w) * (1 - fermi(w)) #empty mode spectral density
    return J1, J2
end

function chain_map(J, N::Int64, D::Float64)
    """calculates family of monic orthogonal polynomials w.r.t the measure J(x) up to the Nth term.
    returns the coefficients alpha and beta from the recurrence relation of the family."""
    supp = (-D, D)
    meas = Measure("bath", J, supp, false, Dict())
    ortho_poly = OrthoPoly("bath_op", N, meas; Nquad=10000)   
    chain = coeffs(ortho_poly)                                  
    Es = chain[1:N,1] #site energies
    ts = sqrt.(chain[1:N,2]) #site hoppings (first term is system hopping)
    return Es, ts
end

function prepare_2body_MPS(N::Int64, sys_state::Vector{String})
    """
    Prepare initial MPS state for 2-body system + Bath
    Sites 1 & 2: System
    Sites 3 to N+2: Filled chain
    Sites N+3 to 2N+2: Empty chain
    """
    
    # Using "Fermion" sites as in your snippet, conserving particle number
    sites = siteinds("Fermion", 2N+2, conserve_qns=true)
    
    # Initialize state array
    # Default: Bath is empty ("Emp"), System is set by sys_state
    states = ["Occ" for _ in 1:2N+2]
    states[4:2:2N+2] .= "Emp"
    
    # Set System States (Site 1 and 2)
    states[1] = sys_state[1]
    states[2] = sys_state[2]
    
    # Optional: Fill half the bath if needed (Fermi sea)
    # for j in 3:2:N_total
    #     states[j] = "Occ"
    # end

    psi0 = MPS(sites, states) 
    return psi0, sites, states
end

function HamiltonianMPO(E_sys, t_sys, V_sys, E1, t1, E2, t2, sites, N)
    
    """
    1 - filled chain
    2 - empty chain

    Fermionic Hamiltonian for 2-body system coupled to a bath chain
    System: Sites 1, 2
    Bath: Sites 3...2N+2
    """

    N_total = length(sites)
    ampo = AutoMPO() 
    
    # --- 1. Internal System Dynamics (Sites 1 & 2) ---
    # Hopping between the two bodies
    add!(ampo, E_sys[1], "N", 1)
    add!(ampo, E_sys[2], "N", 2)

    add!(ampo, t_sys, "Cdag", 1, "C", 2)
    add!(ampo, t_sys, "Cdag", 2, "C", 1)
    
    # Interaction term (e.g., Coulomb repulsion V * n1 * n2)
    add!(ampo, V_sys, "N", 1, "N", 2)
    
    #system-filled chain coupling
    add!(ampo, t1[1], "Cdag", 2, "C", 3)
    add!(ampo, t1[1], "Cdag", 3, "C", 2)

    #system-empty chain coupling
    add!(ampo, t2[1], "Cdag", 2, "C", 4)
    add!(ampo, t2[1], "Cdag", 4, "C", 2)
    
    for j in 1:N
        add!(ampo, E1[j], "N", 2 + 2j-1)
        add!(ampo, E2[j], "N", 2 + 2j)
    end
    for j in 2:N
        add!(ampo, t1[j], "Cdag", 2j-1, "C", 2j+1 )
        add!(ampo, t1[j], "Cdag", 2j+1, "C", 2j-1)
        add!(ampo, t2[j], "Cdag", 2j, "C", 2j+2)
        add!(ampo, t2[j], "Cdag", 2j+2, "C", 2j)
    end
    return MPO(ampo, sites)
end

function evolve_MPS(psi0::MPS, H::MPO, dt::Float64, tmax::Float64)
    """Time evolve MPS with Hamiltonian MPO using TDVP"""
    psi = psi0
    ts = collect(dt:dt:tmax)
    len = length(ts)
    nSys1 = zeros(len)
    nSys2 = zeros(len)
    num = zeros(2N+1)
    for k in 1:len
        psi = tdvp(H, -im*dt, psi; nsite=2, outputlevel=1, mindim=1, maxdim=1000) #time_step=dt, nsweeps=sweeps, order=2)
        nSys1[k] = expect(psi, "N"; sites=1)[1]
        nSys2[k] = expect(psi, "N"; sites=2)[1]
        println("timestep $k of $len complete")
    end
    return nSys1, nSys2
end


In [ ]:
D = 1.0
mu = 0.0
N = 50
g = 0.1
beta = 10.0

E_sys = [0.0, 0.0] #system site energies
t_sys = 0.05 #system site hopping
V_sys = 0.1 #coulomb repulsion

dt = 0.1
tmax = 20.0


J = spectral_func("elliptical", D, g)
J1, J2 = thermofield_transform(J, beta, mu)
E1, t1 = chain_map(J1, N, D)
E2, t2 = chain_map(J2, N, D)

psi0, sites, states = prepare_2body_MPS(N, ["Occ", "Emp"])
H = HamiltonianMPO(E_sys, t_sys, V_sys, E1, t1, E2, t2, sites, N)

nSys1, nSys2 = evolve_MPS(psi0, H, dt, tmax)

In [ ]:
p1 = plot(collect(dt:dt:tmax), nSys1, label="Site 1 Occupation", xlabel="Time", ylabel="Occupation", title="\$ \\beta=$beta, \\mu=$mu, g=$g, t_{sys} = $t_sys, V=$V_sys\$")
plot!(p1, collect(dt:dt:tmax), nSys2, label="Site 2 Occupation", dpi=600)

display(p1)
savefig("twosite.png")